In [23]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64
import webbrowser


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

# import custom CRUD module
from CRUD_module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Set username and password variables for accessing database
username = "aacuser"
password = "apple"
# instantiate a database object
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
# instantiate object to hold database documents
df = pd.DataFrame.from_records(shelter.readAll())

#fig = px.pie(df, names='breed', title='Dog Breeds')
fig = px.pie(data_frame = df, names='breed', title='Dog Breeds', hole = .3)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# set up logo image
image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1(children="Anthony Spedaliere - student"))),
    html.Center(html.B(html.H1('Grazioso Salvaro Search and Rescue Animal Database'))),
    html.Hr(),
    html.A(id='logo_anchor',href='https://www.snhu.edu/', target='_blank',children = [
        html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='logo', height=200))
    ]),
    html.Div(children=[
            dcc.RadioItems(
                id = "radio_items",
   options=[
       {'label': 'Water Rescue', 'value': 'Water Rescue'},
       {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
       {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
       {'label': 'Reset', 'value': 'Reset'},
   ],
   value="Reset"
)                         
    ]),    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),                
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    html.Div([
        dcc.Graph(id = 'pie_chart', figure=fig)        
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# callback function to update the pie chart
@app.callback(
    Output(component_id = 'pie_chart', component_property = "figure"),
    [Input(component_id = 'radio_items', component_property = 'value')]
)
def gen_chart(value):        
    if (value == 'Reset'):
        df = pd.DataFrame.from_records(shelter.readAll())
        fig = px.pie(data_frame = df, names='breed', title='Dog Breeds', hole = .3)
    elif (value == 'Water Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({"breed":{'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 
                                                        'sex_upon_outcome':'Intact Female',"age_upon_outcome":{'$in': ['1 year', '2 years', '3 years']}}))
        fig = px.pie(data_frame = df, names='breed', title='Dog Breeds', hole = .3)
    elif (value == 'Mountain Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({"breed":{'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 
                                                        'sex_upon_outcome':'Intact Male', "age_upon_outcome":{'$in': ['1 year', '2 years', '3 years']}}))
        fig = px.pie(data_frame = df, names='breed', title='Dog Breeds', hole = .3)
    elif (value == 'Disaster Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({"breed":{'$in': ['German Shepherd', 'Doberman Pinscher', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 
                                                        'sex_upon_outcome':'Intact Male', "age_upon_outcome":{'$in': ['1 year', '2 years', '3 years', '4 years', '5 years', '6 years']}}))
        fig = px.pie(data_frame = df, names='breed', title='Dog Breeds', hole = .3)
        
    return fig
        
# callback function to update the data table
@app.callback(
    Output(component_id = 'datatable-id', component_property = 'data'),
    [Input(component_id = 'radio_items', component_property ='value')]
)
def update_table(value):   
    
    if (value == "Reset"):
        df = pd.DataFrame.from_records(shelter.readAll())
    elif (value == 'Water Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({"breed":{'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 
                                                        'sex_upon_outcome':'Intact Female',"age_upon_outcome":{'$in': ['1 year', '2 years', '3 years']}}))
    elif (value == 'Mountain Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({"breed":{'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 
                                                        'sex_upon_outcome':'Intact Male', "age_upon_outcome":{'$in': ['1 year', '2 years', '3 years']}}))
    elif (value == 'Disaster Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({"breed":{'$in': ['German Shepherd', 'Doberman Pinscher', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 
                                                        'sex_upon_outcome':'Intact Male', "age_upon_outcome":{'$in': ['1 year', '2 years', '3 years', '4 years', '5 years', '6 years']}}))
    return df.to_dict('records')
    

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# function to handle the interactive map

"""
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
"""
# function to handle the interactive map


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    df = pd.DataFrame.from_dict(viewData)
    markers = []
    for n in range(len(df.index)):
        markers.append(
            dl.Marker(
                title = df.loc[n].animal_type,
                position = (df.loc[n].location_lat, df.loc[n].location_long),
            )
        )
        
    center_lat = int(df.loc[0].location_lat)
    center_long = int(df.loc[0].location_long)
    
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[center_lat,center_long], zoom=6, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.MarkerClusterGroup(id="markers", children=markers)
        ])
    ]



app